In [286]:
from sets import Set
import string
import copy
from Queue import Queue

In [296]:
alphabet=dict(zip(string.letters,[ord(c)%32 for c in string.letters]))
alphabet_reverse = {}
for x in alphabet:
    alphabet_reverse[alphabet[x]] = x.upper()
    
# alphabet -> num
def letter_to_idx(l):
    return alphabet[l]

# num -> letter
def idx_to_letter(idx):
    return alphabet_reverse[idx]

# build graph
def build_graph(input):
    info = []
    max_idx = 0
    for x in input.replace(' ', '').split(','):
        n_from = letter_to_idx(x[0]) 
        n_to = letter_to_idx(x[1])
        max_idx = max(n_from, n_to, max_idx)
        distance = int(x[2])
        info.append((n_from, n_to, distance))

    graph = [[0 for x in range(max_idx)] for r in range(max_idx)]
    for n_from, n_to, distance in info:
        graph[n_from - 1][n_to - 1] = distance

    return graph

# print as matrix
def print_graph(graph):
    header = " ".join([' '] + [idx_to_letter(x + 1) for x in range(len(graph))])
    print header
    for ri in range(len(graph)):
        node_name = idx_to_letter(ri + 1)
        row = graph[ri]
        row_str = " ".join([node_name] + [str(x) if x != 0 else "0" for x in row])
        print row_str

# node_idx starts from 1
# returns [(node_idx, distance)]
def get_related_nodes(node_idx, graph):
    related_nodes = graph[node_idx - 1]
    return [(i + 1, related_nodes[i]) for i in range(len(related_nodes)) if related_nodes[i] != 0]

# find shortest path using dijkstra algorithm
# returns: shortest_path [node_id], shortest_distance
def dijkstra(start, end, graph):
    
    # letter -> idx
    start = letter_to_idx(start)
    end = letter_to_idx(end)
    
    # {node_id: tent_distance}
    node_tentative_distances = {}
    # {node_id: prev_node_id}
    node_prev = {}
    visited_nodes = Set()

    # compute tentative distance for nodes
    def forward(current_node, current_node_tent_distance):
        # get related nodes
        related_nodes = get_related_nodes(current_node, graph=graph)
        
        # update node tentative distance
        for related_node_idx, distance_to_related_node in related_nodes:
            if related_node_idx not in visited_nodes:
                related_node_tentative_distance = node_tentative_distances.get(related_node_idx)
                if related_node_tentative_distance is None:
                    node_tentative_distances[related_node_idx] = current_node_tent_distance + distance_to_related_node
                    node_prev[related_node_idx] = current_node
                elif related_node_tentative_distance > current_node_tent_distance + distance_to_related_node:
                    # update tentative distance
                    node_tentative_distances[related_node_idx] = current_node_tent_distance + distance_to_related_node
                    node_prev[related_node_idx] = current_node

        # mark current node as visited
        if current_node_tent_distance != 0:
            # do not add start node
            visited_nodes.add(current_node)
        
        # recursion
        for related_node_idx, distance_to_related_node in related_nodes:
            if related_node_idx not in visited_nodes:
                related_node_tentative_distance = node_tentative_distances[related_node_idx]
                forward(related_node_idx, related_node_tentative_distance)
                
    # return None if path not found, or [node_idx]
    def reverse_shortest_path(node_idx):
                        
        def _reverse(node_idx, depth = 0):
            if node_idx == start:
                if start == end and depth == 0:
                    # prevent early exit when start node eq end node
                    return reverse(node_prev[node_idx], depth + 1) + [node_idx]
                else:
                    return [node_idx]
            else:
                return _reverse(node_prev[node_idx], depth + 1) + [node_idx]
                
        try:
            return _reverse(node_idx)
        except Exception as ex:
            return None

    # run
    forward(start, 0)
    shortest_path = reverse_shortest_path(end)    
    shortest_distance = node_tentative_distances.get(end)
    return shortest_path, shortest_distance

# print path
def print_path(path, distance):
    if path is not None:
        path_str = "->".join(idx_to_letter(x) for x in path)
        return '[{}] {}'.format(distance, path_str)
    else:
        return 'PATH NOT FOUND'

# return None if not exist
def get_route_distance(route_str, graph):
    route = route_str.split('-')
    total_distance = 0
    for i in range(len(route) - 1):
        start_idx, end_idx = letter_to_idx(route[i]), letter_to_idx(route[i+1])
        distance = graph[start_idx - 1][end_idx - 1]
        if distance > 0:
            total_distance += distance
        else:
            return None
    return total_distance


In [288]:
class RouteStop(object):
        
    def __init__(self, node_id):
        self.node_id = node_id
        self.next_stop = None
        self.distance = 0
        
    def get_node_id(self):
        return self.node_id

    def set_next_stop(self, next_stop, distance):
        self.next_stop = next_stop
        self.distance = distance
        
    def get_next_stop(self):
        return self.next_stop
        
    def __eq__(self, other):
        return self.node_id == other.node_id and self.distance == other.distance
        
    def get_route(self):
        if self.next_stop is not None:
            return '{} -{}-> {}'.format(idx_to_letter(self.node_id + 1), self.distance, self.next_stop.get_route())
        else:
            return idx_to_letter(self.node_id + 1)
        
class Route(object):
    
    def __init__(self, start_node_id):
        self.start = RouteStop(start_node_id)
        self.end = self.start
        self.route_distance = 0
        self.route_total_stops = 0
        
    def is_empty(self):
        return self.start == self.end
        
    def get_start(self):
        return self.start

    def get_end(self):
        return self.end
    
    def add_stop(self, stop_id, distance):
        next_stop = RouteStop(stop_id)
        self.end.set_next_stop(next_stop, distance)
        self.end = next_stop
        self.route_total_stops += 1
        self.route_distance += distance
        
    def get_route_distance(self):
        return self.route_distance
    
    def get_route_total_stops(self):
        return self.route_total_stops
    
    def print_route(self, print_distance = True):
        print '{} [distance / stops: {} / {}]'.format(self.start.get_route(), self.route_distance, self.route_total_stops)
        
def bsf(start, end, graph, stop_signal):    

    # letter -> idx
    start = letter_to_idx(start) - 1
    end = letter_to_idx(end) - 1
    
    # found routes
    all_routes = []
    
    q = Queue(maxsize=0)
    # start
    q.put(Route(start_node_id=start))
    
    while q.empty() is False:
        route = q.get()
        row = graph[route.get_end().get_node_id()]
        related = [(r, row[r]) for r in range(len(row)) if row[r] != 0]
        for node_id, distance in related:
            new_route = copy.deepcopy(route)            
            new_route.add_stop(node_id, distance)                        
            if stop_signal(new_route) :
                all_routes.append(new_route)
                q.put(new_route)
    
    return [x for x in all_routes if x.get_end().get_node_id() == end]
    
def find_routes_with_exact_stops(start, end, stops):
    all_found_routes = bsf(start, end, graph, lambda route: route.get_route_total_stops() < stops + 1)
    exect_stops = [x for x in all_found_routes if x.get_route_total_stops() == stops]
    return (exect_stops, len(exect_stops))

def find_routes_with_max_stops(start, end, stops):
    all_found_routes = bsf(start, end, graph, lambda route: route.get_route_total_stops() < stops + 1)
    return (all_found_routes, len(all_found_routes))

def find_routes_with_max_distance(start, end, max_distance):
    all_found_routes = bsf(start, end, graph, lambda route: route.get_route_distance() < max_distance)
    return (all_found_routes, len(all_found_routes))


In [274]:
# use cases

In [311]:
# define graph 

input = "AB5, BC4, CD8, DC8, DE6, AD5, CE2, EB3, AE7, CF3, EF4"
graph = build_graph(input)
print_graph(graph)

  A B C D E F
A 0 5 0 5 7 0
B 0 0 4 0 0 0
C 0 0 0 8 2 3
D 0 0 8 0 6 0
E 0 3 0 0 0 4
F 0 0 0 0 0 0


In [312]:
# find shortest path

shortest_path, distance = dijkstra(start='A', end='F', graph=graph)
print_path(shortest_path, distance)

'[11] A->E->F'

In [313]:
# find route distance

distance = get_route_distance('A-B-C', graph)
if distance:
    print 'distance {}'.format(distance)
else:
    print 'NO SUCH ROUTE'

distance 9


In [317]:
# find routes with max stops

r, c = find_routes_with_max_stops('C', 'C', 3)
assert(c == 2)
print c
[i.print_route() for i in r]

2
C -8-> D -8-> C [distance / stops: 16 / 2]
C -2-> E -3-> B -4-> C [distance / stops: 9 / 3]


[None, None]

In [293]:
# find routes with exact stops

r, c = find_routes_with_exact_stops('A', 'C', 4)
assert(c == 3)
print c
[i.print_route() for i in r]

3
A -5-> B -4-> C -8-> D -8-> C [distance / stops: 25 / 4]
A -5-> D -8-> C -8-> D -8-> C [distance / stops: 29 / 4]
A -5-> D -6-> E -3-> B -4-> C [distance / stops: 18 / 4]


[None, None, None]

In [294]:
# find routes with max distance

r, c = find_routes_with_max_distance('C', 'C', 30)
assert(c == 7)
print c
[i.print_route() for i in r]

7
C -8-> D -8-> C [distance / stops: 16 / 2]
C -2-> E -3-> B -4-> C [distance / stops: 9 / 3]
C -8-> D -6-> E -3-> B -4-> C [distance / stops: 21 / 4]
C -8-> D -8-> C -2-> E -3-> B -4-> C [distance / stops: 25 / 5]
C -2-> E -3-> B -4-> C -8-> D -8-> C [distance / stops: 25 / 5]
C -2-> E -3-> B -4-> C -2-> E -3-> B -4-> C [distance / stops: 18 / 6]
C -2-> E -3-> B -4-> C -2-> E -3-> B -4-> C -2-> E -3-> B -4-> C [distance / stops: 27 / 9]


[None, None, None, None, None, None, None]